In [1]:
import numpy as np
import veloxchem as vlx

In [2]:
dimer_xyz = """12

C        0.67759997    0.00000000    -10.0
C       -0.67759997    0.00000000    -10.0
H        1.21655197    0.92414474    -10.0
H        1.21655197   -0.92414474    -10.0
H       -1.21655197   -0.92414474    -10.0
H       -1.21655197    0.92414474    -10.0
C        0.67759997    0.00000000     10.0
C       -0.67759997    0.00000000     10.0
H        1.21655197    0.92414474     10.0
H        1.21655197   -0.92414474     10.0
H       -1.21655197   -0.92414474     10.0
H       -1.21655197    0.92414474     10.0"""

In [3]:
molecule = vlx.Molecule.read_xyz_string(dimer_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31g", ostream=None)

In [4]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_drv.xcfun = "bhandhlyp"
scf_drv.filename = "vlx_nto"

scf_results = scf_drv.compute(molecule, basis)

In [5]:
lreig_drv = vlx.LinearResponseEigenSolver()
lreig_drv.ostream.mute()

lreig_drv.nstates = 2
lreig_drv.nto = True
lreig_drv.nto_cubes = False

lreig_results = lreig_drv.compute(molecule, basis, scf_results)

In [6]:
state = 1 # Python indexing of excited states [0, 1, ...]

In [7]:
print(lreig_results["nto_lambdas"][state])

[3.72500207e-01 3.72474851e-01 2.29242742e-02 2.29210365e-02
 6.24556691e-03 6.24279261e-03 4.88166730e-03 4.88148389e-03
 3.99367521e-03 3.99344265e-03 2.04387353e-03 2.04245691e-03
 4.29082806e-07 4.28246943e-07 3.18969707e-07 3.18046096e-07]


In [8]:
viewer = vlx.OrbitalViewer()
viewer.plot(molecule, basis, "vlx_nto.h5", f"rsp/NTO_S{state + 1}")

Output()

Dropdown(description='Orbital:', index=15, options=(('  1 occ=-0.000 ene=0.000  (alpha HOMO-15)', 0), ('  2 oc…

In [9]:
norb = basis.get_dimension_of_basis()
nocc = molecule.number_of_alpha_electrons()
nvirt = norb - nocc
nexc = nocc * nvirt

print("Number of orbitals:", norb)
print("Number of occupied orbitals:", nocc)
print("Number of unoccupied orbitals:", nvirt)
print("Number of excitations:", nexc)

Number of orbitals: 52
Number of occupied orbitals: 16
Number of unoccupied orbitals: 36
Number of excitations: 576


In [10]:
Xf = lreig_drv.get_full_solution_vector(
    lreig_results["eigenvectors_distributed"][state]
)

Zf = Xf[:nexc]
Yf = Xf[nexc:]
T = np.reshape(Zf - Yf, (nocc, nvirt))
U, L, V = np.linalg.svd(T)

print("Square of diagonal elements:\n", L**2)

Square of diagonal elements:
 [3.72500207e-01 3.72474851e-01 2.29242742e-02 2.29210365e-02
 6.24556691e-03 6.24279261e-03 4.88166730e-03 4.88148389e-03
 3.99367521e-03 3.99344265e-03 2.04387353e-03 2.04245691e-03
 4.29082806e-07 4.28246943e-07 3.18969707e-07 3.18046096e-07]
